In [1]:
#Using Python to analyze my data for the first time. Trying to create a 12-panel figure

In [2]:
#equivalent to figure of seasonal type changes, that can serve as baseline.

In [3]:
import numpy as np
import time
import os

In [4]:
import netCDF4 as nc

In [5]:
ferretpath = "/Users/Siwen/Desktop/ferret/bin"

In [6]:
rdapath = "APHRO_ZH_front_025deg_V1101."

In [7]:
qqq = ferretpath + '/' + rdapath + str(2001) + '.nc'

In [8]:
infile = nc.Dataset(qqq, "r")

In [9]:
infile.dimensions

OrderedDict([('lon',
              <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 72),
             ('lat',
              <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 80),
             ('time',
              <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 365)])

In [10]:
years = np.arange(57) + 1951

In [11]:
seasons = (('Spring',1,60,120), ('Pre-Meiyu',2,121,160),('Meiyu',3,161,200),('Post-Meiyu',4,201,273), \
            ('Fall',5,274,320),('Whole',6,1,365))

In [12]:
P_type_season_yr = np.zeros((57,80,72,6,3))

In [13]:
for i in years:
    
    #Build filename
    filename = ferretpath + '/' + rdapath + str(i) + '.nc'
    print(filename)
    myf = nc.Dataset(filename, 'r')
    
    #Obtain fraction of different types of rainfall
    P_all =  myf.variables['P'][:]
    P_band1 = myf.variables['P_band1'][:]
    P_band2 = myf.variables['P_band2'][:]
    P_band = P_band1 + P_band2
    P_local = myf.variables['P_leftover'][:]
    
    #Find seasonal amounts of each type of rainfall
    for iter in seasons:
    
        P_all_season = P_all[iter[2]-1:iter[3],:,:]
        P_band_season = P_band[iter[2]-1:iter[3],:,:]
        P_local_season = P_local[iter[2]-1:iter[3],:,:]

        P_type_season_yr[i-1951,:,:,iter[1]-1,0] = np.mean(P_all_season,0)
        P_type_season_yr[i-1951,:,:,iter[1]-1,1] = np.mean(P_band_season,0)
        P_type_season_yr[i-1951,:,:,iter[1]-1,2] = np.mean(P_local_season,0)
        
    myf.close()

/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1951.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1952.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1953.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1954.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1955.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1956.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1957.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1958.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1959.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1960.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1961.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1962.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1963.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1964.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_0

In [14]:
#find 57-year mean of yearly values
P_type_season_mean = np.mean(P_type_season_yr,0)
#np.transpose(P_type_season_mean,(1,2,0,3))

In [15]:
saveas = "RDA_type_season.nc"
fullpath = "/Users/Siwen/Desktop/ferret/bin/" + saveas
    
if os.path.exists(fullpath):
    os.remove(fullpath)

#Save as netCDF file
outfile = nc.Dataset(outfilename, "w", format="NETCDF4")
infile = nc.Dataset(qqq, "r")

In [16]:
for dname, the_dim in infile.dimensions.items():
    print(dname, len(the_dim))
    print(the_dim.isunlimited())
    
    #record dimension is unlimited
    outfile.createDimension(dname, len(the_dim) if not \
                            the_dim.isunlimited() else None)
    
#add dimensions for seasons
seasons_dim = outfile.createDimension('seasons', 6)

dim_tuple = ("time","lat","lon","seasons")

#create variables inside netCDF file
P_all_out = outfile.createVariable("P_all_mean", 'f8', dim_tuple)
P_band_out = outfile.createVariable("P_band_mean", 'f8', dim_tuple)
P_local_out = outfile.createVariable("P_local_mean",'f8', dim_tuple)



lon 72
False
lat 80
False
time 365
True


In [17]:
outfile.variables

#Rearrange axes to match intended netCDF output
P_all_out[:] = np.expand_dims(P_type_season_mean[:,:,:,0], axis=0)
P_band_out[:] = np.expand_dims(P_type_season_mean[:,:,:,1], axis=0)
P_local_out[:] = np.expand_dims(P_type_season_mean[:,:,:,2], axis=0)

outfile.close()

#move file to Ferret directory
os.rename("./" + outfilename, "/Users/Siwen/Desktop/ferret/bin/" + outfilename)

In [18]:
outfilename_yrly = "RDA_type_season_yrly.nc"

if os.path.exists(outfilename_yrly):
    os.remove(outfilename_yrly)
    
if os.path.exists("/Users/Siwen/Desktop/ferret/bin/" + outfilename_yrly):
    os.remove("/Users/Siwen/Desktop/ferret/bin/" + outfilename_yrly)
    
outfile_yrly = nc.Dataset(outfilename_yrly, "w", format="NETCDF4")

for dname, the_dim in infile.dimensions.items():
    print(dname, len(the_dim))
    print(the_dim.isunlimited())
    
    #record dimension is unlimited
    outfile_yrly.createDimension(dname, len(the_dim) if not \
                            the_dim.isunlimited() else None)
    
seasons_dim = outfile_yrly.createDimension('seasons', 6)

dim_tuple = ("time","lat","lon","seasons")

#create variables inside netCDF file
P_all_out_yrly = outfile_yrly.createVariable("P_all_mean_yrly", 'f8', dim_tuple)
P_band_out_yrly = outfile_yrly.createVariable("P_band_mean_yrly", 'f8', dim_tuple)
P_local_out_yrly = outfile_yrly.createVariable("P_local_mean_yrly",'f8', dim_tuple)


#Rearrange axes to match intended netCDF output
P_all_out_yrly[:] = P_type_season_yr[:,:,:,:,0]
P_band_out_yrly[:] = P_type_season_yr[:,:,:,:,1]
P_local_out_yrly[:] = P_type_season_yr[:,:,:,:,2]

outfile_yrly.dimensions
outfile_yrly.variables
time.sleep(15)
                                     
outfile_yrly.close()
infile.close()

os.rename(outfilename_yrly, "/Users/Siwen/Desktop/ferret/bin/" + outfilename_yrly)

lon 72
False
lat 80
False
time 365
True
